In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import joblib
import time
import random

from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.preprocessing import MinMaxScaler
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report
from sklearn.svm import SVC

NUM_OF_ROWS = 9000
ATTACK_NAME = 'DoS'

In [2]:
pd.set_option('display.max_columns', None)  # Show all columns

---

In [3]:
# import the attack sample dataset
dosSamples = pd.read_csv('dos_hulk_samples.csv')
dosSamples

,Number of Ports,Average Packet Length,Packet Length Min,Packet Length Max,Packet Length Std,Packet Length Variance,Total Length of Fwd Packet,Fwd Packet Length Max,Fwd Packet Length Mean,Fwd Packet Length Min,Fwd Packet Length Std,Bwd Packet Length Max,Bwd Packet Length Mean,Bwd Packet Length Min,Bwd Packet Length Std,Fwd Segment Size Avg,Bwd Segment Size Avg,Subflow Fwd Bytes,SYN Flag Count,ACK Flag Count,RST Flag Count,Flow Duration,Packets Per Second,IAT Max,IAT Mean,IAT Std
0,1,98.000000,93,103,5.00000,25.000000,39,39,39.000000,39,0.000000,49,49.000000,49,0.000000,0.000000,0.0,0.000000,0,0,0,0.071406,28.008801,0.071406,0.071406,0.000000
1,0,115.000000,115,115,0.00000,0.000000,0,0,0.000000,0,0.000000,81,81.000000,81,0.000000,0.000000,0.0,0.000000,0,0,0,8.089847,1.112506,1.024562,1.011231,0.034083
2,1,127.500000,92,163,35.50000,1260.250000,174,58,58.000000,58,0.000000,129,129.000000,129,0.000000,0.000000,0.0,0.000000,0,0,0,0.004730,1268.502646,0.003781,0.000946,0.001426
3,1,92.000000,92,92,0.00000,0.000000,348,58,58.000000,58,0.000000,0,0.000000,0,0.000000,0.000000,0.0,0.000000,0,0,0,0.004415,1359.065939,0.004412,0.000883,0.001764
4,1,113.448176,54,490,119.39751,14255.765358,792575,456,79.448176,20,119.397510,0,0.000000,0,0.000000,46.704391,0.0,105.578127,2819,5896,1941,3.921313,2544.045958,0.080743,0.000393,0.001147
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
207,1,363.903226,94,1362,438.60625,192375.442248,5892,1308,368.250000,40,479.145007,1305,247.666667,49,381.032749,0.000000,0.0,392.800000,0,0,0,25.923333,1.195834,13.771338,0.864111,2.772981
208,1,127.500000,92,163,35.50000,1260.250000,348,58,58.000000,58,0.000000,129,129.000000,129,0.000000,0.000000,0.0,116.000000,0,0,0,15.059787,0.796824,14.958914,1.369072,4.297560
209,1,92.000000,92,92,0.00000,0.000000,580,58,58.000000,58,0.000000,0,0.000000,0,0.000000,0.000000,0.0,145.000000,0,0,0,15.060015,0.664010,14.957140,1.673335,4.696633
210,1,698.000000,698,698,0.00000,0.000000,4648,664,664.000000,664,0.000000,0,0.000000,0,0.000000,0.000000,0.0,1162.000000,0,0,0,6.758404,1.035747,2.047888,1.126401,0.749836


In [4]:
# dosSamples.drop('IAT Total', axis=1, inplace=True)
# dosSamples = dosSamples.rename(columns={'Average Packet Size': 'Average Packet Length'})
# dosSamples.to_csv('dos_hulk_new_samples2.csv', index=False)

In [5]:
# get all the relevant attack rows from the attack sample dataset
dosSamples = dosSamples[dosSamples['RST Flag Count'] > 20]

In [6]:
# print some general information about the attack samples
print(f'Dataset Shape: {dosSamples.shape}')
dosSamples.reset_index() 
# finding the correlation between the Packet Length Std column to the rest of the columns in order to create new data

Dataset Shape: (22, 26)


,index,Number of Ports,Average Packet Length,Packet Length Min,Packet Length Max,Packet Length Std,Packet Length Variance,Total Length of Fwd Packet,Fwd Packet Length Max,Fwd Packet Length Mean,Fwd Packet Length Min,Fwd Packet Length Std,Bwd Packet Length Max,Bwd Packet Length Mean,Bwd Packet Length Min,Bwd Packet Length Std,Fwd Segment Size Avg,Bwd Segment Size Avg,Subflow Fwd Bytes,SYN Flag Count,ACK Flag Count,RST Flag Count,Flow Duration,Packets Per Second,IAT Max,IAT Mean,IAT Std
0,4,1,113.448176,54,490,119.397510,14255.765358,792575,456,79.448176,20,119.397510,0,0.0,0,0.0,46.704391,0.0,105.578127,2819,5896,1941,3.921313,2544.045958,0.080743,0.000393,0.001147
1,6,1,111.399578,54,492,118.554999,14055.287716,770745,458,77.399578,20,118.554999,0,0.0,0,0.0,45.408817,0.0,97.897244,2411,5932,2245,4.324354,2302.771733,0.041172,0.000434,0.001078
2,17,1,114.086974,54,490,122.233724,14941.083217,799268,456,80.086974,20,122.233724,0,0.0,0,0.0,48.782766,0.0,102.260491,1790,6418,2440,3.992678,2499.575369,0.052491,0.000400,0.000896
3,23,1,115.067034,54,488,123.774303,15320.077971,809049,454,81.067034,20,123.774303,0,0.0,0,0.0,50.176052,0.0,95.881607,1477,6596,2554,5.106365,1954.423561,0.100327,0.000512,0.001462
4,29,1,113.853212,54,488,122.117633,14912.716396,798053,454,79.853212,20,122.117633,0,0.0,0,0.0,48.652892,0.0,103.724071,1767,6383,2478,4.209765,2374.004139,0.047036,0.000421,0.000916
5,32,1,117.572573,54,489,125.957802,15865.367856,834890,455,83.572573,20,125.957802,0,0.0,0,0.0,52.133133,0.0,103.893728,1520,6990,2320,4.390662,2275.283341,0.070261,0.000440,0.001078
6,36,1,116.129676,54,490,122.376793,14976.079363,818915,456,82.129676,20,122.376793,0,0.0,0,0.0,49.261559,0.0,104.147908,2318,6829,1677,10.304537,967.632019,4.278063,0.001034,0.044491
7,42,1,110.064299,54,489,116.821153,13647.181842,758285,455,76.064299,20,116.821153,0,0.0,0,0.0,43.861270,0.0,104.432585,2632,5751,2172,3.477777,2866.486457,0.037094,0.000349,0.000705
8,46,1,106.599037,54,490,111.687568,12474.112822,723885,456,72.599037,20,111.687568,0,0.0,0,0.0,39.632233,0.0,85.092865,3146,5531,1760,9.178418,1086.352771,2.364858,0.000921,0.027002
9,54,1,104.720705,54,489,109.305360,11947.661642,701903,455,70.720705,20,109.305360,0,0.0,0,0.0,38.039194,0.0,84.070308,3115,5297,1848,16.610516,597.513051,1.989192,0.001674,0.032656


In [7]:
# find the columns that we need to synthesis data for to produce an attack dataset
columnsToGather = dosSamples.replace(0, np.nan) #replace all 0 values with null
columnsToGather = columnsToGather.dropna(how="all", axis=1).columns.tolist() #remove all columns where there are null values
columnsToGather #left with all columns that the values are not 0 (be know for a fact that the data is consistant and there are not missing values in the rows)

['Number of Ports',
 'Average Packet Length',
 'Packet Length Min',
 'Packet Length Max',
 'Packet Length Std',
 'Packet Length Variance',
 'Total Length of Fwd Packet',
 'Fwd Packet Length Max',
 'Fwd Packet Length Mean',
 'Fwd Packet Length Min',
 'Fwd Packet Length Std',
 'Fwd Segment Size Avg',
 'Subflow Fwd Bytes',
 'SYN Flag Count',
 'ACK Flag Count',
 'RST Flag Count',
 'Flow Duration',
 'Packets Per Second',
 'IAT Max',
 'IAT Mean',
 'IAT Std']

In [8]:
# find an approximate minimum and maximum values of each column and save that data into a dictionary
MinMaxDict = {col: (dosSamples[col].min() * 0.85, dosSamples[col].max() * 1.15) for col in columnsToGather}
MinMaxDict['Number of Ports'] = (1, 1)
MinMaxDict 

{'Number of Ports': (1, 1),
 'Average Packet Length': (np.float64(84.26142361541321),
  np.float64(135.20845845845844)),
 'Packet Length Min': (np.float64(45.9), np.float64(62.099999999999994)),
 'Packet Length Max': (np.float64(413.95), np.float64(565.8)),
 'Packet Length Std': (np.float64(85.69969727608765),
  np.float64(144.85147217068953)),
 'Packet Length Variance': (np.float64(8640.515427309489),
  np.float64(18245.17303479656)),
 'Total Length of Fwd Packet': (np.float64(446822.05),
  np.float64(960123.4999999999)),
 'Fwd Packet Length Max': (np.float64(385.05), np.float64(526.6999999999999)),
 'Fwd Packet Length Mean': (np.float64(55.36142361541321),
  np.float64(96.10845845845844)),
 'Fwd Packet Length Min': (np.float64(17.0), np.float64(23.0)),
 'Fwd Packet Length Std': (np.float64(85.69969727608765),
  np.float64(144.85147217068953)),
 'Fwd Segment Size Avg': (np.float64(26.95029736092182),
  np.float64(59.9531031031031)),
 'Subflow Fwd Bytes': (np.float64(60.07364130434783)

In [9]:
# change values to int for the columns that should be int (because when calculating approximate values they will be turned into floats)
intColumns = ['SYN Flag Count', 'ACK Flag Count', 'RST Flag Count']
for key, val in MinMaxDict.items():
    if key in intColumns:
        MinMaxDict[key] = tuple([int(v) for v in val])
MinMaxDict

{'Number of Ports': (1, 1),
 'Average Packet Length': (np.float64(84.26142361541321),
  np.float64(135.20845845845844)),
 'Packet Length Min': (np.float64(45.9), np.float64(62.099999999999994)),
 'Packet Length Max': (np.float64(413.95), np.float64(565.8)),
 'Packet Length Std': (np.float64(85.69969727608765),
  np.float64(144.85147217068953)),
 'Packet Length Variance': (np.float64(8640.515427309489),
  np.float64(18245.17303479656)),
 'Total Length of Fwd Packet': (np.float64(446822.05),
  np.float64(960123.4999999999)),
 'Fwd Packet Length Max': (np.float64(385.05), np.float64(526.6999999999999)),
 'Fwd Packet Length Mean': (np.float64(55.36142361541321),
  np.float64(96.10845845845844)),
 'Fwd Packet Length Min': (np.float64(17.0), np.float64(23.0)),
 'Fwd Packet Length Std': (np.float64(85.69969727608765),
  np.float64(144.85147217068953)),
 'Fwd Segment Size Avg': (np.float64(26.95029736092182),
  np.float64(59.9531031031031)),
 'Subflow Fwd Bytes': (np.float64(60.07364130434783)

### Creating the dataset

In [10]:
# creating an empty dataframe before adding values to it
dosDataset = pd.DataFrame(np.zeros((NUM_OF_ROWS, len(dosSamples.columns))), columns=dosSamples.columns)
dosDataset.head(3)

,Number of Ports,Average Packet Length,Packet Length Min,Packet Length Max,Packet Length Std,Packet Length Variance,Total Length of Fwd Packet,Fwd Packet Length Max,Fwd Packet Length Mean,Fwd Packet Length Min,Fwd Packet Length Std,Bwd Packet Length Max,Bwd Packet Length Mean,Bwd Packet Length Min,Bwd Packet Length Std,Fwd Segment Size Avg,Bwd Segment Size Avg,Subflow Fwd Bytes,SYN Flag Count,ACK Flag Count,RST Flag Count,Flow Duration,Packets Per Second,IAT Max,IAT Mean,IAT Std
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [11]:
# adding zeros to all columns that should not have any values
zeroColumns = [col for col in dosSamples.columns if col not in columnsToGather]
for col in zeroColumns:
    dosDataset[col] = int(0)
zeroColumns

['Bwd Packet Length Max',
 'Bwd Packet Length Mean',
 'Bwd Packet Length Min',
 'Bwd Packet Length Std',
 'Bwd Segment Size Avg']

In [12]:
dosDataset.head(3)

,Number of Ports,Average Packet Length,Packet Length Min,Packet Length Max,Packet Length Std,Packet Length Variance,Total Length of Fwd Packet,Fwd Packet Length Max,Fwd Packet Length Mean,Fwd Packet Length Min,Fwd Packet Length Std,Bwd Packet Length Max,Bwd Packet Length Mean,Bwd Packet Length Min,Bwd Packet Length Std,Fwd Segment Size Avg,Bwd Segment Size Avg,Subflow Fwd Bytes,SYN Flag Count,ACK Flag Count,RST Flag Count,Flow Duration,Packets Per Second,IAT Max,IAT Mean,IAT Std
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0,0,0,0.0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0,0,0,0.0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0,0,0,0.0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


### Calculating the correct corelation between columns

### First group

In [13]:
# finding the correlation between the SYN Flag Count column to the rest of the columns in order to create new data
firstCorrelation = ['SYN Flag Count', 'ACK Flag Count', 'RST Flag Count']
independent_col = dosSamples[firstCorrelation[0]].values.reshape(-1, 1) #column 'SYN Flag Count'
dependent_cols = dosSamples[firstCorrelation[1:]].values #the rest of the columns that are not zeros

# find the scaling factors using least squares function
scaling_factors = np.linalg.lstsq(independent_col, dependent_cols, rcond=None)[0]

scaling_factors = [(name,factor) for name, factor in zip(firstCorrelation[1:], scaling_factors.flatten())]
for val in scaling_factors:
    print(val)

('ACK Flag Count', np.float64(1.9518394723650978))
('RST Flag Count', np.float64(0.6629111435788864))


In [14]:
dosDataset['SYN Flag Count'] = np.random.randint(MinMaxDict['SYN Flag Count'][0]*0.85, MinMaxDict['SYN Flag Count'][1]*1.15, NUM_OF_ROWS)

for index, row in dosDataset.iterrows():
    for col, factor in scaling_factors: #iterating over all rows we need to add values
        # calculate the value we want to add into the dataset in the given row
        delta = random.uniform(factor * 0.1, factor * 0.2) # select a delta
        updatedFactor = factor + random.choice([-1, 1]) * delta
        dosDataset.loc[index, col] = int(row['SYN Flag Count'] * updatedFactor)

In [15]:
dosDataset.head(10)

,Number of Ports,Average Packet Length,Packet Length Min,Packet Length Max,Packet Length Std,Packet Length Variance,Total Length of Fwd Packet,Fwd Packet Length Max,Fwd Packet Length Mean,Fwd Packet Length Min,Fwd Packet Length Std,Bwd Packet Length Max,Bwd Packet Length Mean,Bwd Packet Length Min,Bwd Packet Length Std,Fwd Segment Size Avg,Bwd Segment Size Avg,Subflow Fwd Bytes,SYN Flag Count,ACK Flag Count,RST Flag Count,Flow Duration,Packets Per Second,IAT Max,IAT Mean,IAT Std
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0,0,0,0.0,0,0.0,2136,3751.0,1614.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0,0,0,0.0,0,0.0,3156,5330.0,1729.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0,0,0,0.0,0,0.0,3604,5947.0,2040.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0,0,0,0.0,0,0.0,4725,7724.0,3717.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0,0,0,0.0,0,0.0,3362,7646.0,2571.0,0.0,0.0,0.0,0.0,0.0
5,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0,0,0,0.0,0,0.0,3928,8692.0,2133.0,0.0,0.0,0.0,0.0,0.0
6,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0,0,0,0.0,0,0.0,2375,5490.0,1741.0,0.0,0.0,0.0,0.0,0.0
7,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0,0,0,0.0,0,0.0,5318,11828.0,4170.0,0.0,0.0,0.0,0.0,0.0
8,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0,0,0,0.0,0,0.0,4304,7427.0,2295.0,0.0,0.0,0.0,0.0,0.0
9,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0,0,0,0.0,0,0.0,3420,7919.0,2667.0,0.0,0.0,0.0,0.0,0.0


### Second group

In [16]:
# Generate random values for the 'Flow Duration' column
randValues = np.random.uniform(MinMaxDict['Flow Duration'][0]*0.9, MinMaxDict['Flow Duration'][1]*1.05, size=NUM_OF_ROWS)

# Assign the random values
dosDataset['Flow Duration'] = randValues

In [17]:
# finding the correlation between the Number of Ports column to the rest of the columns in order to create new data
secondCorrelation = ['Flow Duration', 'Packets Per Second', 'IAT Max', 'IAT Mean', 'IAT Std']
independent_col = dosSamples[secondCorrelation[0]].values.reshape(-1, 1) #column 'Number of Ports'
dependent_cols = dosSamples[secondCorrelation[1:]].values #the rest of the columns that are not zeros

# find the scaling factors using least squares function
scaling_factors = np.linalg.lstsq(independent_col, dependent_cols, rcond=None)[0]

scaling_factors = [(name,factor) for name, factor in zip(secondCorrelation[1:], scaling_factors.flatten())]
for val in scaling_factors:
    print(val)

('Packets Per Second', np.float64(29.47374840796544))
('IAT Max', np.float64(0.28959035080285817))
('IAT Mean', np.float64(0.00010689125614908151))
('IAT Std', np.float64(0.0035898854422333083))


In [18]:
durationToPacketsCorr = [x * y for x, y in zip(dosSamples['Flow Duration'].values, dosSamples['Packets Per Second'].values)]
durationToPacketsCorr = np.mean(durationToPacketsCorr)
durationToPacketsCorr

np.float64(9827.272727272728)

In [19]:
#iterating over all rows we need to add values
for index, row in dosDataset.iterrows():
    for col, factor in scaling_factors: 
        # calculate a random small delta of the factor for adding some randomness
        if col == 'Packets Per Second':
            delta = random.uniform(durationToPacketsCorr * 0.035, durationToPacketsCorr * 0.175) # select a delta
            updatedFactor = durationToPacketsCorr + random.choice([-1, 1]) * delta
            dosDataset.loc[index, col] = updatedFactor / row['Flow Duration']
        else:
            if col == 'IAT Std' or col == 'IAT Max':
                delta = random.uniform(factor * 0.7, factor * 0.95)
                updatedFactor = factor + random.choices([-1, 1], weights=[2, 1], k=1)[0] * delta
            else:
                delta = random.uniform(factor * 0.05, factor * 0.15) # select a delta
                updatedFactor = factor + random.choice([-1, 1]) * delta
            dosDataset.loc[index, col] = row['Flow Duration'] * updatedFactor

In [20]:
dosDataset.describe()

,Number of Ports,Average Packet Length,Packet Length Min,Packet Length Max,Packet Length Std,Packet Length Variance,Total Length of Fwd Packet,Fwd Packet Length Max,Fwd Packet Length Mean,Fwd Packet Length Min,Fwd Packet Length Std,Bwd Packet Length Max,Bwd Packet Length Mean,Bwd Packet Length Min,Bwd Packet Length Std,Fwd Segment Size Avg,Bwd Segment Size Avg,Subflow Fwd Bytes,SYN Flag Count,ACK Flag Count,RST Flag Count,Flow Duration,Packets Per Second,IAT Max,IAT Mean,IAT Std
count,9000.0,9000.0,9000.0,9000.0,9000.0,9000.0,9000.0,9000.0,9000.0,9000.0,9000.0,9000.0,9000.0,9000.0,9000.0,9000.0,9000.0,9000.0,9000.000000,9000.000000,9000.000000,9000.000000,9000.000000,9000.000000,9000.000000,9000.000000
mean,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,3225.850000,6292.019111,2138.909889,26.149625,610.054072,5.534102,0.002797,0.068048
std,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1248.591012,2647.016301,896.729288,13.554086,597.275127,7.257655,0.001488,0.089559
min,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1066.000000,1666.000000,577.000000,2.662929,164.135286,0.042747,0.000248,0.000494
25%,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2127.750000,4102.750000,1389.000000,14.436760,260.864839,0.821144,0.001517,0.010244
50%,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,3231.000000,6147.000000,2106.500000,26.236594,374.463501,1.873595,0.002785,0.023654
75%,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,4310.250000,8217.000000,2796.250000,37.727349,683.334231,8.014408,0.004006,0.095604
max,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,5385.000000,12539.000000,4276.000000,49.570897,4276.012297,27.867658,0.006057,0.344502


In [21]:
x = dosDataset[dosDataset['Flow Duration']<10]
x[x['Flow Duration']>5][0:40]

,Number of Ports,Average Packet Length,Packet Length Min,Packet Length Max,Packet Length Std,Packet Length Variance,Total Length of Fwd Packet,Fwd Packet Length Max,Fwd Packet Length Mean,Fwd Packet Length Min,Fwd Packet Length Std,Bwd Packet Length Max,Bwd Packet Length Mean,Bwd Packet Length Min,Bwd Packet Length Std,Fwd Segment Size Avg,Bwd Segment Size Avg,Subflow Fwd Bytes,SYN Flag Count,ACK Flag Count,RST Flag Count,Flow Duration,Packets Per Second,IAT Max,IAT Mean,IAT Std
12,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0,0,0,0.0,0,0.0,4611,10254.0,3511.0,6.363752,1614.153856,3.524715,0.000729,0.003702
14,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0,0,0,0.0,0,0.0,3693,5807.0,2107.0,8.878453,1273.755665,4.756182,0.000872,0.006943
16,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0,0,0,0.0,0,0.0,2857,4958.0,1698.0,8.653887,1202.400765,0.160617,0.000972,0.060344
24,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0,0,0,0.0,0,0.0,3292,7552.0,2538.0,5.765677,1804.238637,3.193895,0.000666,0.002119
45,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0,0,0,0.0,0,0.0,2954,6560.0,1699.0,9.522282,925.731156,0.621273,0.000949,0.006635
50,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0,0,0,0.0,0,0.0,4203,6688.0,2259.0,5.218498,2125.532087,0.257994,0.000615,0.001174
61,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0,0,0,0.0,0,0.0,1891,3031.0,1057.0,5.397225,1669.264022,0.335398,0.000544,0.005482
64,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0,0,0,0.0,0,0.0,1723,3024.0,1019.0,7.193182,1279.914994,3.746041,0.000716,0.001923
70,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0,0,0,0.0,0,0.0,4860,8311.0,2600.0,6.846698,1331.407283,3.854440,0.000785,0.042226
78,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0,0,0,0.0,0,0.0,4126,9572.0,2352.0,6.044664,1457.414927,3.408876,0.000613,0.003222


In [22]:
x = dosDataset[dosDataset['Flow Duration']<50.5]
x[x['Flow Duration']>25][20:40]

,Number of Ports,Average Packet Length,Packet Length Min,Packet Length Max,Packet Length Std,Packet Length Variance,Total Length of Fwd Packet,Fwd Packet Length Max,Fwd Packet Length Mean,Fwd Packet Length Min,Fwd Packet Length Std,Bwd Packet Length Max,Bwd Packet Length Mean,Bwd Packet Length Min,Bwd Packet Length Std,Fwd Segment Size Avg,Bwd Segment Size Avg,Subflow Fwd Bytes,SYN Flag Count,ACK Flag Count,RST Flag Count,Flow Duration,Packets Per Second,IAT Max,IAT Mean,IAT Std
34,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0,0,0,0.0,0,0.0,1192,2082.0,682.0,48.036541,192.358273,3.776766,0.004534,0.025051
35,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0,0,0,0.0,0,0.0,3952,9214.0,2307.0,26.865159,336.182952,14.543537,0.002568,0.164823
36,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0,0,0,0.0,0,0.0,4126,6843.0,3278.0,34.247117,264.312710,2.012017,0.004173,0.012612
38,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0,0,0,0.0,0,0.0,3118,5214.0,2290.0,42.095623,212.761189,0.770467,0.004779,0.022975
39,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0,0,0,0.0,0,0.0,5301,9117.0,3038.0,30.578054,343.812718,16.211763,0.003454,0.016847
40,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0,0,0,0.0,0,0.0,4782,8277.0,3721.0,27.483271,315.433971,13.624008,0.003306,0.022679
41,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0,0,0,0.0,0,0.0,3922,8620.0,2277.0,47.096477,216.057438,3.897789,0.005316,0.016535
43,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0,0,0,0.0,0,0.0,4300,9675.0,3155.0,46.594068,185.877829,23.964194,0.005522,0.019904
46,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0,0,0,0.0,0,0.0,3595,6039.0,2853.0,42.627955,258.543941,3.074592,0.003976,0.018424
47,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0,0,0,0.0,0,0.0,1776,2776.0,1358.0,42.203131,248.914896,0.799293,0.004976,0.007637


In [23]:
dosDataset[dosDataset['Flow Duration']<6][:20]

,Number of Ports,Average Packet Length,Packet Length Min,Packet Length Max,Packet Length Std,Packet Length Variance,Total Length of Fwd Packet,Fwd Packet Length Max,Fwd Packet Length Mean,Fwd Packet Length Min,Fwd Packet Length Std,Bwd Packet Length Max,Bwd Packet Length Mean,Bwd Packet Length Min,Bwd Packet Length Std,Fwd Segment Size Avg,Bwd Segment Size Avg,Subflow Fwd Bytes,SYN Flag Count,ACK Flag Count,RST Flag Count,Flow Duration,Packets Per Second,IAT Max,IAT Mean,IAT Std
24,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0,0,0,0.0,0,0.0,3292,7552.0,2538.0,5.765677,1804.238637,3.193895,0.000666,0.002119
50,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0,0,0,0.0,0,0.0,4203,6688.0,2259.0,5.218498,2125.532087,0.257994,0.000615,0.001174
61,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0,0,0,0.0,0,0.0,1891,3031.0,1057.0,5.397225,1669.264022,0.335398,0.000544,0.005482
95,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0,0,0,0.0,0,0.0,2865,4712.0,1675.0,3.044432,3510.835389,1.667839,0.000349,0.020831
97,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0,0,0,0.0,0,0.0,4800,8395.0,3778.0,5.518411,1974.961349,3.036088,0.000552,0.002765
139,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0,0,0,0.0,0,0.0,4937,7748.0,3681.0,4.320300,2168.722571,0.285355,0.000403,0.027506
172,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0,0,0,0.0,0,0.0,2989,6853.0,1681.0,3.371693,2451.645130,1.824174,0.000385,0.003173
181,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0,0,0,0.0,0,0.0,2272,3791.0,1657.0,5.025161,2055.231283,0.303099,0.000497,0.033527
183,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0,0,0,0.0,0,0.0,3246,5509.0,2567.0,3.502277,3286.787597,0.225374,0.000340,0.023709
208,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0,0,0,0.0,0,0.0,1999,4316.0,1145.0,3.726024,2346.264930,2.077349,0.000447,0.025883


### Third group

In [24]:
# finding the correlation between the Packet Length Std column to the rest of the columns in order to create new data
firstCorrelation = ['Average Packet Length', 'Packet Length Std', 'Packet Length Variance', 'Total Length of Fwd Packet', 
                    'Fwd Packet Length Mean', 'Fwd Packet Length Std', 'Fwd Segment Size Avg', 'Subflow Fwd Bytes']
independent_col = dosSamples[firstCorrelation[0]].values.reshape(-1, 1) #column 'Packet Length Std'
dependent_cols = dosSamples[firstCorrelation[1:]].values #the rest of the columns that are not zeros

# find the scaling factors using least squares function
scaling_factors = np.linalg.lstsq(independent_col, dependent_cols, rcond=None)[0]

scaling_factors = [(name,factor) for name, factor in zip(firstCorrelation[1:], scaling_factors.flatten())]
for val in scaling_factors:
    print(val)

('Packet Length Std', np.float64(1.053481965176044))
('Packet Length Variance', np.float64(122.15926993685173))
('Total Length of Fwd Packet', np.float64(6798.663741787297))
('Fwd Packet Length Mean', np.float64(0.6901644819681063))
('Fwd Packet Length Std', np.float64(1.053481965176044))
('Fwd Segment Size Avg', np.float64(0.39446115648863966))
('Subflow Fwd Bytes', np.float64(0.8079142101649653))


In [25]:
dosDataset['Average Packet Length'] = np.random.uniform(MinMaxDict['Average Packet Length'][0]*0.85, MinMaxDict['Average Packet Length'][1]*1.15, NUM_OF_ROWS)

for index, row in dosDataset.iterrows():
    for col, factor in scaling_factors: #iterating over all rows we need to add values
        # calculate the value we want to add into the dataset in the given row
        delta = random.uniform(factor * 0.1, factor * 0.2) # select a delta
        updatedFactor = factor + random.choice([-1, 1]) * delta
        dosDataset.loc[index, col] = row['Average Packet Length'] * updatedFactor

In [26]:
x = dosDataset[dosDataset['Average Packet Length'] > 114]
x[x['Average Packet Length'] < 120].tail(20)

,Number of Ports,Average Packet Length,Packet Length Min,Packet Length Max,Packet Length Std,Packet Length Variance,Total Length of Fwd Packet,Fwd Packet Length Max,Fwd Packet Length Mean,Fwd Packet Length Min,Fwd Packet Length Std,Bwd Packet Length Max,Bwd Packet Length Mean,Bwd Packet Length Min,Bwd Packet Length Std,Fwd Segment Size Avg,Bwd Segment Size Avg,Subflow Fwd Bytes,SYN Flag Count,ACK Flag Count,RST Flag Count,Flow Duration,Packets Per Second,IAT Max,IAT Mean,IAT Std
8848,0.0,115.810427,0.0,0.0,100.877054,12377.296145,875645.839844,0.0,95.355647,0.0,104.649273,0,0,0,0,53.221701,0,77.407208,3940,6732.0,3017.0,7.220540,1222.024976,0.563640,0.000811,0.050393
8860,0.0,117.398673,0.0,0.0,101.226238,12329.029143,917565.795645,0.0,91.985961,0.0,104.239903,0,0,0,0,41.022265,0,77.406992,3952,6737.0,3127.0,42.305385,222.783833,2.289676,0.004184,0.037231
8866,0.0,117.596097,0.0,0.0,140.811219,11750.841079,650125.296906,0.0,72.164642,0.0,144.704793,0,0,0,0,51.514491,0,104.729536,2609,4581.0,1477.0,34.022482,246.962700,2.172618,0.003863,0.031024
8868,0.0,119.465475,0.0,0.0,102.550231,12491.459088,905082.380922,0.0,92.536660,0.0,145.801255,0,0,0,0,56.065117,0,112.770468,3523,5632.0,2756.0,39.437553,210.015532,21.781487,0.003824,0.040635
8872,0.0,117.778903,0.0,0.0,141.712130,16587.388877,695155.655608,0.0,70.630132,0.0,102.038386,0,0,0,0,37.650452,0,107.196034,5242,11609.0,3008.0,7.079395,1511.209788,0.261200,0.000845,0.007478
8874,0.0,119.461597,0.0,0.0,105.451982,12378.717865,724140.187267,0.0,67.779681,0.0,150.044340,0,0,0,0,41.428696,0,115.561998,3571,6189.0,2068.0,41.521861,213.197838,2.955272,0.004184,0.042270
8900,0.0,119.972482,0.0,0.0,105.731873,16615.699828,928260.156905,0.0,93.200117,0.0,139.589739,0,0,0,0,40.488058,0,109.836579,2713,4509.0,2001.0,35.378091,297.899084,18.432222,0.004123,0.216111
8901,0.0,118.467689,0.0,0.0,103.804672,16836.495271,713264.159737,0.0,66.622813,0.0,110.051985,0,0,0,0,52.343292,0,109.958688,1968,3406.0,1435.0,30.769742,357.210193,1.272722,0.003028,0.025029
8904,0.0,119.417823,0.0,0.0,143.260227,17237.352943,703434.828373,0.0,98.866813,0.0,102.699261,0,0,0,0,54.341147,0,80.691086,4353,7414.0,2478.0,38.003203,244.806630,18.833137,0.003485,0.009122
8912,0.0,117.832193,0.0,0.0,105.856942,16105.188922,897615.289125,0.0,72.964100,0.0,147.855620,0,0,0,0,37.718541,0,85.054455,2413,4232.0,1350.0,30.444185,278.124070,1.847620,0.002828,0.207728


### Independant Columns

In [27]:
# finding the correlation between the Packet Length Std column to the rest of the columns in order to create new data
independantColumns = ['Packet Length Min', 'Packet Length Max', 'Fwd Packet Length Max', 'Fwd Packet Length Min']

for col in independantColumns:
    dosDataset[col] = (np.random.uniform(MinMaxDict[col][0]*0.85, MinMaxDict[col][1]*1.15, NUM_OF_ROWS)).astype(int)

dosDataset

,Number of Ports,Average Packet Length,Packet Length Min,Packet Length Max,Packet Length Std,Packet Length Variance,Total Length of Fwd Packet,Fwd Packet Length Max,Fwd Packet Length Mean,Fwd Packet Length Min,Fwd Packet Length Std,Bwd Packet Length Max,Bwd Packet Length Mean,Bwd Packet Length Min,Bwd Packet Length Std,Fwd Segment Size Avg,Bwd Segment Size Avg,Subflow Fwd Bytes,SYN Flag Count,ACK Flag Count,RST Flag Count,Flow Duration,Packets Per Second,IAT Max,IAT Mean,IAT Std
0,0.0,122.732182,52,395,113.717378,17669.833706,701192.525877,447,73.007141,19,107.460374,0,0,0,0,55.310812,0,113.421218,2136,3751.0,1614.0,46.741987,191.064599,2.536167,0.004269,0.036281
1,0.0,84.548568,68,458,98.324506,8845.414714,673233.815459,430,64.481368,20,100.549931,0,0,0,0,29.067911,0,80.570711,3156,5330.0,1729.0,34.088260,248.414724,17.121381,0.003457,0.227921
2,0.0,114.326391,63,516,105.634843,15614.435230,627899.116409,358,65.320358,20,141.939177,0,0,0,0,39.286842,0,109.310751,3604,5947.0,2040.0,25.290619,370.085436,2.174509,0.003045,0.174498
3,0.0,115.041997,70,428,106.156074,16242.060630,886047.439213,542,69.751509,19,138.636120,0,0,0,0,53.862080,0,82.370620,4725,7724.0,3717.0,40.792700,221.678186,21.507850,0.003871,0.034309
4,0.0,84.805313,54,358,77.582985,11636.365538,514222.394234,415,67.259000,23,72.824645,0,0,0,0,37.594392,0,60.769726,3362,7646.0,2571.0,15.957232,661.580946,8.199922,0.001851,0.009501
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8995,0.0,83.663450,56,538,101.611333,9056.972820,677566.620083,600,66.053751,19,75.918081,0,0,0,0,37.415366,0,77.628228,1546,3362.0,904.0,29.751053,307.732903,15.724419,0.003493,0.193586
8996,0.0,131.947715,42,493,161.306560,13223.345818,776048.318131,540,73.243198,15,112.885384,0,0,0,0,45.553541,0,123.484509,3418,5953.0,2523.0,6.323273,1440.733339,0.395467,0.000588,0.003208
8997,0.0,90.652158,61,592,114.438212,12679.534534,517042.238860,550,69.014972,15,78.635178,0,0,0,0,41.565194,0,63.823947,1144,2567.0,886.0,16.796089,492.587702,9.452091,0.001906,0.013760
8998,0.0,106.140395,57,553,130.999101,11458.149793,804748.907292,518,63.219039,24,97.701177,0,0,0,0,46.215722,0,74.440237,2265,5011.0,1332.0,2.690597,3814.382212,0.181278,0.000249,0.018587


### Adding labels and verifiying the dataset

In [28]:
# adding number of ports and a label to the dataset
dosDataset['Number of Ports'] = np.full(shape=NUM_OF_ROWS, fill_value=1, dtype=int)
dosDataset['Label'] = ATTACK_NAME

In [29]:
dosDataset

,Number of Ports,Average Packet Length,Packet Length Min,Packet Length Max,Packet Length Std,Packet Length Variance,Total Length of Fwd Packet,Fwd Packet Length Max,Fwd Packet Length Mean,Fwd Packet Length Min,Fwd Packet Length Std,Bwd Packet Length Max,Bwd Packet Length Mean,Bwd Packet Length Min,Bwd Packet Length Std,Fwd Segment Size Avg,Bwd Segment Size Avg,Subflow Fwd Bytes,SYN Flag Count,ACK Flag Count,RST Flag Count,Flow Duration,Packets Per Second,IAT Max,IAT Mean,IAT Std,Label
0,1,122.732182,52,395,113.717378,17669.833706,701192.525877,447,73.007141,19,107.460374,0,0,0,0,55.310812,0,113.421218,2136,3751.0,1614.0,46.741987,191.064599,2.536167,0.004269,0.036281,DoS
1,1,84.548568,68,458,98.324506,8845.414714,673233.815459,430,64.481368,20,100.549931,0,0,0,0,29.067911,0,80.570711,3156,5330.0,1729.0,34.088260,248.414724,17.121381,0.003457,0.227921,DoS
2,1,114.326391,63,516,105.634843,15614.435230,627899.116409,358,65.320358,20,141.939177,0,0,0,0,39.286842,0,109.310751,3604,5947.0,2040.0,25.290619,370.085436,2.174509,0.003045,0.174498,DoS
3,1,115.041997,70,428,106.156074,16242.060630,886047.439213,542,69.751509,19,138.636120,0,0,0,0,53.862080,0,82.370620,4725,7724.0,3717.0,40.792700,221.678186,21.507850,0.003871,0.034309,DoS
4,1,84.805313,54,358,77.582985,11636.365538,514222.394234,415,67.259000,23,72.824645,0,0,0,0,37.594392,0,60.769726,3362,7646.0,2571.0,15.957232,661.580946,8.199922,0.001851,0.009501,DoS
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8995,1,83.663450,56,538,101.611333,9056.972820,677566.620083,600,66.053751,19,75.918081,0,0,0,0,37.415366,0,77.628228,1546,3362.0,904.0,29.751053,307.732903,15.724419,0.003493,0.193586,DoS
8996,1,131.947715,42,493,161.306560,13223.345818,776048.318131,540,73.243198,15,112.885384,0,0,0,0,45.553541,0,123.484509,3418,5953.0,2523.0,6.323273,1440.733339,0.395467,0.000588,0.003208,DoS
8997,1,90.652158,61,592,114.438212,12679.534534,517042.238860,550,69.014972,15,78.635178,0,0,0,0,41.565194,0,63.823947,1144,2567.0,886.0,16.796089,492.587702,9.452091,0.001906,0.013760,DoS
8998,1,106.140395,57,553,130.999101,11458.149793,804748.907292,518,63.219039,24,97.701177,0,0,0,0,46.215722,0,74.440237,2265,5011.0,1332.0,2.690597,3814.382212,0.181278,0.000249,0.018587,DoS


In [30]:
dosSamples.describe()

,Number of Ports,Average Packet Length,Packet Length Min,Packet Length Max,Packet Length Std,Packet Length Variance,Total Length of Fwd Packet,Fwd Packet Length Max,Fwd Packet Length Mean,Fwd Packet Length Min,Fwd Packet Length Std,Bwd Packet Length Max,Bwd Packet Length Mean,Bwd Packet Length Min,Bwd Packet Length Std,Fwd Segment Size Avg,Bwd Segment Size Avg,Subflow Fwd Bytes,SYN Flag Count,ACK Flag Count,RST Flag Count,Flow Duration,Packets Per Second,IAT Max,IAT Mean,IAT Std
count,22.0,22.000000,22.0,22.000000,22.000000,22.000000,22.000000,22.000000,22.000000,22.0,22.000000,22.0,22.0,22.0,22.0,22.000000,22.0,22.000000,22.000000,22.000000,22.000000,22.000000,22.000000,22.000000,22.000000,22.000000
mean,1.0,109.541236,54.0,489.272727,115.336644,13344.256197,743456.272727,455.272727,75.541236,20.0,115.336644,0.0,0.0,0.0,0.0,43.067228,0.0,88.236516,2693.954545,5715.863636,1978.409091,14.334490,1273.183737,3.416355,0.001497,0.042374
std,0.0,4.723316,0.0,1.031957,6.610693,1503.443014,66624.894863,1.031957,4.723316,0.0,6.610693,0.0,0.0,0.0,0.0,5.371901,0.0,12.216166,687.545218,696.524932,410.466594,11.577552,890.026163,4.604678,0.001294,0.055395
min,1.0,99.131087,54.0,487.000000,100.823173,10165.312267,525673.000000,453.000000,65.131087,20.0,100.823173,0.0,0.0,0.0,0.0,31.706232,0.0,70.674872,1477.000000,4081.000000,1100.000000,3.477777,196.600053,0.037094,0.000349,0.000705
25%,1.0,106.700900,54.0,489.000000,111.827633,12505.478401,710284.500000,455.000000,72.700900,20.0,111.827633,0.0,0.0,0.0,0.0,39.767573,0.0,78.726481,2327.500000,5252.000000,1742.750000,4.569588,465.317607,0.072882,0.000458,0.001226
50%,1.0,109.955870,54.0,489.000000,116.146897,13490.556399,757735.000000,455.000000,75.955870,20.0,116.146897,0.0,0.0,0.0,0.0,43.302417,0.0,86.390381,2894.000000,5704.000000,1904.000000,8.535311,1174.495592,1.109663,0.000856,0.015098
75%,1.0,113.082372,54.0,490.000000,120.085389,14420.658297,788886.750000,456.000000,79.082372,20.0,120.085389,0.0,0.0,0.0,0.0,46.672949,0.0,101.169679,3118.000000,6270.250000,2301.250000,21.099838,2195.068396,5.987090,0.002152,0.080042
max,1.0,117.572573,54.0,492.000000,125.957802,15865.367856,834890.000000,458.000000,83.572573,20.0,125.957802,0.0,0.0,0.0,0.0,52.133133,0.0,105.578127,4075.000000,6990.000000,2662.000000,41.052888,2866.486457,16.314272,0.005087,0.200255


In [31]:
dosDataset.describe()

,Number of Ports,Average Packet Length,Packet Length Min,Packet Length Max,Packet Length Std,Packet Length Variance,Total Length of Fwd Packet,Fwd Packet Length Max,Fwd Packet Length Mean,Fwd Packet Length Min,Fwd Packet Length Std,Bwd Packet Length Max,Bwd Packet Length Mean,Bwd Packet Length Min,Bwd Packet Length Std,Fwd Segment Size Avg,Bwd Segment Size Avg,Subflow Fwd Bytes,SYN Flag Count,ACK Flag Count,RST Flag Count,Flow Duration,Packets Per Second,IAT Max,IAT Mean,IAT Std
count,9000.0,9000.000000,9000.000000,9000.000000,9000.000000,9000.000000,9.000000e+03,9000.000000,9000.000000,9000.000000,9000.000000,9000.0,9000.0,9000.0,9000.0,9000.000000,9000.0,9000.000000,9000.000000,9000.000000,9000.000000,9000.000000,9000.000000,9000.000000,9000.000000,9000.000000
mean,1.0,113.355077,54.847444,501.789111,119.470268,13851.003195,7.689537e+05,465.453444,78.230080,19.961333,119.514689,0.0,0.0,0.0,0.0,44.638462,0.0,91.722929,3225.850000,6292.019111,2138.909889,26.149625,610.054072,5.534102,0.002797,0.068048
std,0.0,24.109189,9.327442,86.584454,31.210186,3653.067696,2.020420e+05,80.240520,20.766172,3.490666,31.702280,0.0,0.0,0.0,0.0,11.669801,0.0,24.309575,1248.591012,2647.016301,896.729288,13.554086,597.275127,7.257655,0.001488,0.089559
min,1.0,71.627007,39.000000,351.000000,60.629961,7070.348322,3.913890e+05,327.000000,39.604070,14.000000,60.506978,0.0,0.0,0.0,0.0,22.703708,0.0,46.451167,1066.000000,1666.000000,577.000000,2.662929,164.135286,0.042747,0.000248,0.000494
25%,1.0,92.591398,47.000000,426.000000,95.372610,11039.294812,6.136620e+05,396.000000,62.050361,17.000000,95.155241,0.0,0.0,0.0,0.0,35.799825,0.0,73.062784,2127.750000,4102.750000,1389.000000,14.436760,260.864839,0.821144,0.001517,0.010244
50%,1.0,113.150692,55.000000,504.000000,116.564329,13503.876219,7.503226e+05,465.000000,76.095804,20.000000,116.623375,0.0,0.0,0.0,0.0,43.639948,0.0,89.250996,3231.000000,6147.000000,2106.500000,26.236594,374.463501,1.873595,0.002785,0.023654
75%,1.0,133.920389,63.000000,577.000000,139.193199,16130.387709,8.964067e+05,534.000000,91.286893,23.000000,139.644043,0.0,0.0,0.0,0.0,51.949013,0.0,107.093875,4310.250000,8217.000000,2796.250000,37.727349,683.334231,8.014408,0.004006,0.095604
max,1.0,155.475648,71.000000,650.000000,195.432566,22760.037957,1.261100e+06,605.000000,128.333935,26.000000,196.031671,0.0,0.0,0.0,0.0,73.144309,0.0,150.177490,5385.000000,12539.000000,4276.000000,49.570897,4276.012297,27.867658,0.006057,0.344502


In [32]:
dosDataset.shape

(9000, 27)

In [ ]:
# save the dataset
# dosDataset.to_csv('dos_hulk_dataset_new.csv', index=False)

---